# Phys 581 Winter 2019
# Report #1: Cluster computing
## Alexander Hickey, 10169582

Note that the contents of this notebook were created and tested on an Ubuntu 18.04.2 machine, using Python 2.7.16. The parallel python library used in this notebook is not supported in Python 3+.

In [2]:
import sys
sys.version
sys.path

['',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python27.zip',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/plat-linux2',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/lib-tk',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/lib-old',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/lib-dynload',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/site-packages',
 '/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/site-packages/IPython/extensions',
 '/home/alexander.hickey/.ipython']

In [3]:
import pp
import os
import time
import paramiko
import getpass

### Introduction

As explored in Assignment 5, the Multiprocessing module offers a convenient means of parallelizing the execution of a function across multiple input values. This allows one to side-step the notorious Python global interpreter lock, by spawning subprocesses rather than threads. The simplest, and arguably most useful function of the multiprocessing package is the Pool object, which spawns multiple independent computations in parallel. This is ideal in cases where one is performing repetetive computations that are independent of one another, for example, performing a function on some large list of values, where the results are independent of one another. 

One of the downsides of any simple implementation of the multiprocessing module, is that the efficiency of performing tasks in parallel is limited by the number of physical cores present on the device. Often times however, one has access to many computers that are a part of the same network. It is therefore of interest to be able to run tasks in parallel across an entire network of CPUs, rather than just on a single device. This technique of getting multiple computers across a network to work together is known as cluster computing.

This report will explore the construction and efficiency of a computing cluster, using the Parallel Python package.

Install using
    
    conda install -c geneko pp
    conda install -c anaconda paramiko
    

In [4]:
#Retrieve username and prompt user for password
user = getpass.getuser()
passw = getpass.getpass('Password for '+user+': ')

Password for alexander.hickey: ········


In [5]:
#Set server path and network base IP
path = '/home/%s/anaconda3/envs/Python27/bin/ppserver.py'%user
IP_base = '136.159.54.'
path

'/home/alexander.hickey/anaconda3/envs/Python27/bin/ppserver.py'

In [6]:
hostlist = []
for j in range(30,40):
    
    hostname = IP_base+str(j)
    
    try:
        with paramiko.SSHClient() as ssh:
            ssh = paramiko.SSHClient()
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.connect(hostname, username=user, password=passw,timeout=.1)
            ssh.close()

            hostlist.append(hostname)
        
    except:
        #print 'Couldn\'t connect to %s' % hostname
        None

print 'Successfully connected to %s hosts:' %len(hostlist) 
print(hostlist)

/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.curve, Q_S_bytes
/home/alexander.hickey/anaconda3/envs/Python27/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:111: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.p

Successfully connected to 9 hosts:
['136.159.54.30', '136.159.54.31', '136.159.54.33', '136.159.54.34', '136.159.54.35', '136.159.54.36', '136.159.54.37', '136.159.54.38', '136.159.54.39']


In [7]:
command = 'conda activate Python27 && ppserver.py'

In [8]:
cnt = 0
for host in hostlist:
    
    try:
        with paramiko.SSHClient() as ssh:
        #ssh = paramiko.SSHClient()
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.connect(host, username=user, password=passw,timeout=2)
            ssh.exec_command(command)
            #ssh.exec_command(command2)
            #ssh.close()
            cnt += 1

    except:
        None
        
print 'ppserver.py executed on %s hosts'%cnt

ppserver.py executed on 9 hosts


In [9]:
ppservers= tuple(hostlist)
job_server = pp.Server(ppservers = ppservers)

In [11]:
job_server.get_active_nodes()

{'136.159.54.30:60000': 4,
 '136.159.54.31:60000': 4,
 '136.159.54.33:60000': 4,
 '136.159.54.34:60000': 4,
 '136.159.54.35:60000': 4,
 '136.159.54.36:60000': 4,
 '136.159.54.37:60000': 4,
 '136.159.54.38:60000': 4,
 '136.159.54.39:60000': 4,
 'local': 4}

In [12]:
command = 'pkill -f ppserver.py'
cnt = 0
for host in hostlist:
    
    try:
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(host, username=user, password=passw,timeout=2)
        ssh.exec_command(command)
        ssh.close()
        cnt += 1
        
    except:
        None
        
print 'disconnected from %s hosts'%cnt

disconnected from 9 hosts


In [ ]:
def f(x):
    
    time.sleep(.5)
    
    return None

t0 = time.time()

inputs = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16)
for i in inputs:
    
    f(i)
    
t = time.time()-t0
    
print('Time: ' +str(t))

In [ ]:
import time
def f(x):
    
    time.sleep(1.0)
    
    return x

t0 = time.time()

inputs = (1,2,3,4,5,6,7,8)
jobs = [(i, job_server.submit(f,(i,), modules = ("time",))) for i in inputs]
for i, job in jobs:
    print i

t = time.time()-t0
    
print('Time: ' +str(t))
job_server.print_stats()

In [ ]:
#Spawn node
os.system("konsole -e ssh munin 'conda activate Python27 && ppserver.py'")

#Kill process
os.system("konsole -e ssh munin 'pkill -u alexander.hickey'")